In [ ]:
import numpy as np

# Datei einlesen und Daten als np array mit gelösche Laufnummern zuückgeben.
def readfiles(myfile):
    data = []
    with open(myfile, "r") as file: 
        for line in file:
            parts = line.strip().split()
            data.append(parts)
    
    data =np.array(data)[ : , 1: ]
        
    return data
        

# In ein NumPy-Array umwandeln
#Laufende Nummer der Instanz von 0 bis 1999 löschen, weil uns nichts bringt.
r1=readfiles("runtimes.mp80")

print(r1)
print (r1.shape)

[['03.05.2024-17:11:29' '03.05.2024-17:11:36' '03.05.2024-17:11:58']
 ['03.05.2024-17:11:58' '03.05.2024-17:12:25' '03.05.2024-17:13:03']
 ['03.05.2024-17:13:03' '03.05.2024-17:13:10' '03.05.2024-17:13:27']
 ...
 ['05.05.2024-01:42:48' '05.05.2024-01:42:56' '05.05.2024-01:43:36']
 ['05.05.2024-01:43:36' '05.05.2024-01:43:44' '05.05.2024-01:44:26']
 ['05.05.2024-01:44:26' '05.05.2024-01:44:33' '05.05.2024-01:44:44']]
(2000, 3)


In [22]:
from datetime import datetime

r1_times = np.array([[datetime.strptime(x, "%d.%m.%Y-%H:%M:%S") for x in row] for row in r1])

print(r1_times)


[[datetime.datetime(2024, 5, 3, 17, 11, 29)
  datetime.datetime(2024, 5, 3, 17, 11, 36)
  datetime.datetime(2024, 5, 3, 17, 11, 58)]
 [datetime.datetime(2024, 5, 3, 17, 11, 58)
  datetime.datetime(2024, 5, 3, 17, 12, 25)
  datetime.datetime(2024, 5, 3, 17, 13, 3)]
 [datetime.datetime(2024, 5, 3, 17, 13, 3)
  datetime.datetime(2024, 5, 3, 17, 13, 10)
  datetime.datetime(2024, 5, 3, 17, 13, 27)]
 ...
 [datetime.datetime(2024, 5, 5, 1, 42, 48)
  datetime.datetime(2024, 5, 5, 1, 42, 56)
  datetime.datetime(2024, 5, 5, 1, 43, 36)]
 [datetime.datetime(2024, 5, 5, 1, 43, 36)
  datetime.datetime(2024, 5, 5, 1, 43, 44)
  datetime.datetime(2024, 5, 5, 1, 44, 26)]
 [datetime.datetime(2024, 5, 5, 1, 44, 26)
  datetime.datetime(2024, 5, 5, 1, 44, 33)
  datetime.datetime(2024, 5, 5, 1, 44, 44)]]
